In [34]:
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sgcrf import SparseGaussianCRF
import pprint

In [35]:
df = pd.read_csv('fservice.csv')
c = ['Anonymous','AssignTo', 'RequestType', 'RequestSource','CD','Direction', 'ActionTaken', 'APC' ,'AddressVerified']
d = ['Latitude', 'Longitude']

In [36]:
def elapsedHours(createdDate, closedDate): 
    """
    - Returns elapsedHours between two dates
    """
    
    #Convert date/time strings to datetime objects
    created = datetime.strptime(createdDate, '%m/%d/%Y %I:%M:%S %p')
    closed = datetime.strptime(closedDate, '%m/%d/%Y %I:%M:%S %p')
    
    #Difference between createdDate and closedDate
    diff = closed - created
    
    #Compute elapsedHours
    days, seconds = diff.days, diff.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    minutes = minutes + seconds/60
    hours = hours + minutes/60

    return hours


def CreateTrainSet(df, predictor_num):
    """
    - Creates training set.
    - Accepts df with training data.
    - predictor_num is the specified number of previous elapsedHour values used to predict 
    the next one.   
    """
    X_train = []
    X_test = []
    
    #Create training set 
    for i in range(0,df.shape[0] - predictor_num): 
        X_train.append(df['ElapsedHours'][i:i + predictor_num].values)
        X_test.append(df['ElapsedHours'][i + predictor_num])
    
    return np.asarray(X_train), np.asarray(X_test)
    

def CreateTestSet(df, predictor_num):
    """
    - Similar to CreateTrainSet. Creates testing set.
    - Accepts df with testing data.
    - predictor_num is the specified number of previous elapsedHour values used to predict 
    the next one. Same value as before   
    """
    y_train = []
    y_test = []
    
    #Create testing Set 
    for i in range(0,df.shape[0] - predictor_num): 
        y_train.append(df['ElapsedHours'][i:i + predictor_num].values)
        y_test.append(df['ElapsedHours'][i + predictor_num])

    return np.asarray(y_train), np.asarray(y_test)

def metrics(y_hat,y):
    """
    Return rmse and mae
    """
    rmse_days = np.sqrt(mean_squared_error(y_hat, y, multioutput='raw_values'))
    mae = mean_absolute_error(y_hat, y)
    
    return rmse_days, mae


def CleanedFrame(df): 
    """
    Creates elapsedHours column, and converts
    CreatedDate and ClosedDate columns to datetime objects, Sorts dataframe 
    chronologically by CreatedDate
    """
    df['ElapsedHours'] = df.apply(lambda x: elapsedHours(x['CreatedDate'],x['ClosedDate']),axis=1)
    df['ClosedDate'] = df['ClosedDate'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
    df['CreatedDate'] = df['CreatedDate'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
    df = df.sort_values(by="CreatedDate")

    return df


def preprocessing(df, start_date, end_date):
    """
    Filters dataframe by specified start and end_dates and runs CleanedFrame on it.  
    """ 
    
    #Filter dataframe by dates 
    df['Just Date'] = df['Just Date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
    df = df[(df['Just Date'] >= start_date) & (df['Just Date'] <= end_date)]
    df = CleanedFrame(df)
    
    # exclude more than 11 days (264 hours)
    df = df[df['ElapsedDays'] <= 11]

    return df

#Model
def lacer(df, df1, train_start_date, train_end_date, test_start_date, test_end_date, request_type, CD, predictor_num, n_iter=10000): #Once model is ready, replace df with csv
    """
    Trains 3 GCRF models on data from specified CD, Request Type, and Owner which is assigned to fulfill request. 
    Uses specified start and end dates for training and testing to creat train and test sets. 
    """
    print("preprocessing...")
    #Create Training and Testing Sets
    dftrain = preprocessing(df, train_start_date, train_end_date)
    dftrain = dftrain.reset_index(drop = True)
    dftest = preprocessing(df1, test_start_date, test_end_date)
    dftest = dftest.reset_index(drop = True)

    #Reserve test set for training on all 3 models. 
    y_train, y_test = CreateTestSet(dftest, predictor_num)
    y_test = y_test.reshape((-1, 1))


## 3 Models
    #Model1: CD
    modelCD = SparseGaussianCRF(lamL=0.1, lamT=0.1, n_iter=n_iter)
    dftrainCD = dftrain[dftrain['CD'] == CD].reset_index(drop = True)

    X_trainCD, X_testCD = CreateTrainSet(dftrainCD, predictor_num)
    X_testCD = X_testCD.reshape((-1, 1))
    modelCD.fit(X_trainCD, X_testCD)

    y_predCD = modelCD.predict(y_train)

    #Model2: Request_type
    modelRT = SparseGaussianCRF(lamL=0.1, lamT=0.1, n_iter=n_iter)
    dftrainRT = dftrain[dftrain['RequestType'] == request_type].reset_index(drop = True)

    X_trainRT, X_testRT = CreateTrainSet(dftrainRT, predictor_num)
    X_testRT = X_testRT.reshape((-1, 1))

    modelRT.fit(X_trainRT, X_testRT)

    y_predRT = modelRT.predict(y_train)

    #Model3: Owner
    #modelOwner = SparseGaussianCRF(lamL=0.1, lamT=0.1, n_iter=10000)
    #dftrainOwner = dftrain[dftrain['Owner'] == owner].reset_index(drop = True)

    #X_trainOwner, X_testOwner = CreateTrainSet(dftrainOwner, predictor_num)
    #X_testOwner = X_testOwner.reshape((-1, 1))

    #modelOwner.fit(X_trainOwner, X_testOwner)

    #y_predOwner = modelOwner.predict(y_train)

#Average out all predictions
    y_predFinal = (y_predCD + y_predRT)/2

#Return metrics 
    return metrics(y_predFinal, y_test)

In [37]:
df1 = df.copy()

In [38]:
pd.unique(df['Owner'])

array(['ITA', 'OCB', 'BOS', 'BSL', 'BSS', 'LADWP', 'BOE', 'RAP', 'LASAN',
       nan, 'LAAS', 'LADOT'], dtype=object)

In [39]:
pd.unique(df['RequestType'])

array(['Other', 'Graffiti Removal', 'Bulky Items', 'Dead Animal Removal',
       'Metal/Household Appliances', 'Single Streetlight Issue',
       'Illegal Dumping Pickup', 'Feedback', 'Multiple Streetlight Issue',
       'Report Water Waste', 'Homeless Encampment', 'Electronic Waste'],
      dtype=object)

In [40]:
sorted(pd.unique(df['CD']))

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 nan]

In [49]:
def cer_date_optimizer_only_range(search_start_date : str, search_week_range : tuple, test_weeks : int, df, df1, predictor_num, cd, request_type):    
    """Run the model for training sets of varying size to see which size is best."""
    # search_week_range is a tuple of which week ranges to check
    output = []
    for interval_size in range(search_week_range[0], search_week_range[1]):
        # iterate over the numbers of weeks of interval_size that fit within search_start_date and search_end_date
        print("making date ranges...")
        this_train_start_date = datetime.strptime(search_start_date,'%Y-%m-%d')
        this_train_end_date = timedelta(weeks=interval_size) + this_train_start_date
        this_test_start_date = timedelta(days=1) + this_train_end_date
        this_test_end_date = timedelta(days=1, weeks=test_weeks) + this_test_start_date
            
        this_train_start_date = this_train_start_date.strftime('%Y-%m-%d')
        this_train_end_date = this_train_end_date.strftime('%Y-%m-%d')
        this_test_start_date = this_test_start_date.strftime('%Y-%m-%d')
        this_test_end_date = this_test_end_date.strftime('%Y-%m-%d')    
        
        print("copying dataframes...")
        df_new = df.copy(deep=True)
        df1_new = df1.copy(deep=True)
            
        print(f"running lacer...")
        this_metric = str(lacer(df_new, df1_new, this_train_start_date, this_train_end_date, this_test_start_date, this_test_end_date,
                                        request_type, cd, predictor_num))
        output.append("interval: " + str(interval_size) + ", " +  "train_start: " + str(this_train_start_date) + ', ' + "train_end: " + str(this_train_end_date) +  ': ' + this_metric)
        print(f"output={output}")
    return output

In [47]:
cer_date_list = cer_date_optimizer_only_range('2018-01-01', (2, 10), 4, df, df1, 50, 5, "Bulky Items")

making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 2, train_start: 2018-01-01, train_end: 2018-01-15: (array([326.86794258]), 48.4462740174892)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 2, train_start: 2018-01-01, train_end: 2018-01-15: (array([326.86794258]), 48.4462740174892)', 'interval: 3, train_start: 2018-01-01, train_end: 2018-01-22: (array([337.10175507]), 48.01492008905323)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


...

/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 2, train_start: 2018-01-01, train_end: 2018-01-15: (array([326.86794258]), 48.4462740174892)', 'interval: 3, train_start: 2018-01-01, train_end: 2018-01-22: (array([337.10175507]), 48.01492008905323)', 'interval: 4, train_start: 2018-01-01, train_end: 2018-01-29: (array([342.67187832]), 48.02464540247928)', 'interval: 5, train_start: 2018-01-01, train_end: 2018-02-05: (array([340.09735782]), 47.90958288837643)', 'interval: 6, train_start: 2018-01-01, train_end: 2018-02-12: (array([307.66796207]), 46.862360055416566)', 'interval: 7, train_start: 2018-01-01, train_end: 2018-02-19: (array([69.07746231]), 43.25413236918686)', 'interval: 8, train_start: 2018-01-01, train_end: 2018-02-26: (array([62.2416482]), 42.874477774120656)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 2, train_start: 2018-01-01, train_end: 2018-01-15: (array([326.86794258]), 48.4462740174892)', 'interval: 3, train_start: 2018-01-01, train_end: 2018-01-22: (array([337.10175507]), 48.01492008905323)', 'interval: 4, train_start: 2018-01-01, train_end: 2018-01-29: (array([342.67187832]), 48.02464540247928)', 'interval: 5, train_start: 2018-01-01, train_end: 2018-02-05: (array([340.09735782]), 47.90958288837643)', 'interval: 6, train_start: 2018-01-01, train_end: 2018-02-12: (array([307.66796207]), 46.862360055416566)', 'interval: 7, train_start: 2018-01-01, train_end: 2018-02-19: (array([69.07746231]), 43.25413236918686)', 'interval: 8, train_start: 2018-01-01, train_end: 2018-02-26: (array([62.2416482]), 42.874477774120656)', 'interval: 9, train_start: 2018-01-01, train_end: 2018-03-05: (array([55.24822819]), 43.10015011010714)']


['interval: 2, train_start: 2018-01-01, train_end: 2018-01-15: (array([326.86794258]), 48.4462740174892)',
 'interval: 3, train_start: 2018-01-01, train_end: 2018-01-22: (array([337.10175507]), 48.01492008905323)',
 'interval: 4, train_start: 2018-01-01, train_end: 2018-01-29: (array([342.67187832]), 48.02464540247928)',
 'interval: 5, train_start: 2018-01-01, train_end: 2018-02-05: (array([340.09735782]), 47.90958288837643)',
 'interval: 6, train_start: 2018-01-01, train_end: 2018-02-12: (array([307.66796207]), 46.862360055416566)',
 'interval: 7, train_start: 2018-01-01, train_end: 2018-02-19: (array([69.07746231]), 43.25413236918686)',
 'interval: 8, train_start: 2018-01-01, train_end: 2018-02-26: (array([62.2416482]), 42.874477774120656)',
 'interval: 9, train_start: 2018-01-01, train_end: 2018-03-05: (array([55.24822819]), 43.10015011010714)']

In [50]:
cer_date_list_10_to_20 = cer_date_optimizer_only_range('2018-01-01', (10, 21), 4, df, df1, 50, 5, "Bulky Items")

making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 10, train_start: 2018-01-01, train_end: 2018-03-12: (array([55.02282092]), 43.020781489936404)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 10, train_start: 2018-01-01, train_end: 2018-03-12: (array([55.02282092]), 43.020781489936404)', 'interval: 11, train_start: 2018-01-01, train_end: 2018-03-19: (array([54.44514797]), 42.61426258919832)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


...

output=['interval: 10, train_start: 2018-01-01, train_end: 2018-03-12: (array([55.02282092]), 43.020781489936404)', 'interval: 11, train_start: 2018-01-01, train_end: 2018-03-19: (array([54.44514797]), 42.61426258919832)', 'interval: 12, train_start: 2018-01-01, train_end: 2018-03-26: (array([54.67025344]), 42.896498417764086)', 'interval: 13, train_start: 2018-01-01, train_end: 2018-04-02: (array([54.62546137]), 43.13815437741188)', 'interval: 14, train_start: 2018-01-01, train_end: 2018-04-09: (array([61.40533134]), 43.04008638220296)', 'interval: 15, train_start: 2018-01-01, train_end: 2018-04-16: (array([62.26158428]), 43.81966009488144)', 'interval: 16, train_start: 2018-01-01, train_end: 2018-04-23: (array([61.94708396]), 43.67250673910408)', 'interval: 17, train_start: 2018-01-01, train_end: 2018-04-30: (array([62.10634918]), 43.88356649325955)', 'interval: 18, train_start: 2018-01-01, train_end: 2018-05-07: (array([54.5793698]), 43.459063963669216)']
making date ranges...
copyi

/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 10, train_start: 2018-01-01, train_end: 2018-03-12: (array([55.02282092]), 43.020781489936404)', 'interval: 11, train_start: 2018-01-01, train_end: 2018-03-19: (array([54.44514797]), 42.61426258919832)', 'interval: 12, train_start: 2018-01-01, train_end: 2018-03-26: (array([54.67025344]), 42.896498417764086)', 'interval: 13, train_start: 2018-01-01, train_end: 2018-04-02: (array([54.62546137]), 43.13815437741188)', 'interval: 14, train_start: 2018-01-01, train_end: 2018-04-09: (array([61.40533134]), 43.04008638220296)', 'interval: 15, train_start: 2018-01-01, train_end: 2018-04-16: (array([62.26158428]), 43.81966009488144)', 'interval: 16, train_start: 2018-01-01, train_end: 2018-04-23: (array([61.94708396]), 43.67250673910408)', 'interval: 17, train_start: 2018-01-01, train_end: 2018-04-30: (array([62.10634918]), 43.88356649325955)', 'interval: 18, train_start: 2018-01-01, train_end: 2018-05-07: (array([54.5793698]), 43.459063963669216)', 'interval: 19, train_start:

/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 10, train_start: 2018-01-01, train_end: 2018-03-12: (array([55.02282092]), 43.020781489936404)', 'interval: 11, train_start: 2018-01-01, train_end: 2018-03-19: (array([54.44514797]), 42.61426258919832)', 'interval: 12, train_start: 2018-01-01, train_end: 2018-03-26: (array([54.67025344]), 42.896498417764086)', 'interval: 13, train_start: 2018-01-01, train_end: 2018-04-02: (array([54.62546137]), 43.13815437741188)', 'interval: 14, train_start: 2018-01-01, train_end: 2018-04-09: (array([61.40533134]), 43.04008638220296)', 'interval: 15, train_start: 2018-01-01, train_end: 2018-04-16: (array([62.26158428]), 43.81966009488144)', 'interval: 16, train_start: 2018-01-01, train_end: 2018-04-23: (array([61.94708396]), 43.67250673910408)', 'interval: 17, train_start: 2018-01-01, train_end: 2018-04-30: (array([62.10634918]), 43.88356649325955)', 'interval: 18, train_start: 2018-01-01, train_end: 2018-05-07: (array([54.5793698]), 43.459063963669216)', 'interval: 19, train_start:

In [52]:
pprint.pprint(cer_date_list_10_to_20)

['interval: 10, train_start: 2018-01-01, train_end: 2018-03-12: '
 '(array([55.02282092]), 43.020781489936404)',
 'interval: 11, train_start: 2018-01-01, train_end: 2018-03-19: '
 '(array([54.44514797]), 42.61426258919832)',
 'interval: 12, train_start: 2018-01-01, train_end: 2018-03-26: '
 '(array([54.67025344]), 42.896498417764086)',
 'interval: 13, train_start: 2018-01-01, train_end: 2018-04-02: '
 '(array([54.62546137]), 43.13815437741188)',
 'interval: 14, train_start: 2018-01-01, train_end: 2018-04-09: '
 '(array([61.40533134]), 43.04008638220296)',
 'interval: 15, train_start: 2018-01-01, train_end: 2018-04-16: '
 '(array([62.26158428]), 43.81966009488144)',
 'interval: 16, train_start: 2018-01-01, train_end: 2018-04-23: '
 '(array([61.94708396]), 43.67250673910408)',
 'interval: 17, train_start: 2018-01-01, train_end: 2018-04-30: '
 '(array([62.10634918]), 43.88356649325955)',
 'interval: 18, train_start: 2018-01-01, train_end: 2018-05-07: '
 '(array([54.5793698]), 43.459063963

In [53]:
cer_date_list_21_to_30 = cer_date_optimizer_only_range('2018-01-01', (21, 30), 4, df, df1, 50, 5, "Bulky Items")

making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 21, train_start: 2018-01-01, train_end: 2018-05-28: (array([95.16250911]), 43.46016518949431)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 21, train_start: 2018-01-01, train_end: 2018-05-28: (array([95.16250911]), 43.46016518949431)', 'interval: 22, train_start: 2018-01-01, train_end: 2018-06-04: (array([96.61961472]), 44.145179889437806)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


...

output=['interval: 21, train_start: 2018-01-01, train_end: 2018-05-28: (array([95.16250911]), 43.46016518949431)', 'interval: 22, train_start: 2018-01-01, train_end: 2018-06-04: (array([96.61961472]), 44.145179889437806)', 'interval: 23, train_start: 2018-01-01, train_end: 2018-06-11: (array([95.67498684]), 43.74815735594257)', 'interval: 24, train_start: 2018-01-01, train_end: 2018-06-18: (array([110.56939455]), 44.326671874930646)', 'interval: 25, train_start: 2018-01-01, train_end: 2018-06-25: (array([77.78828816]), 43.859590175430206)', 'interval: 26, train_start: 2018-01-01, train_end: 2018-07-02: (array([76.99935791]), 43.11735617365377)', 'interval: 27, train_start: 2018-01-01, train_end: 2018-07-09: (array([81.25511843]), 44.0228364860092)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 21, train_start: 2018-01-01, train_end: 2018-05-28: (array([95.16250911]), 43.46016518949431)', 'interval: 22, train_start: 2018-01-01, train_end: 2018-06-04: (array([96.61961472]), 44.145179889437806)', 'interval: 23, train_start: 2018-01-01, train_end: 2018-06-11: (array([95.67498684]), 43.74815735594257)', 'interval: 24, train_start: 2018-01-01, train_end: 2018-06-18: (array([110.56939455]), 44.326671874930646)', 'interval: 25, train_start: 2018-01-01, train_end: 2018-06-25: (array([77.78828816]), 43.859590175430206)', 'interval: 26, train_start: 2018-01-01, train_end: 2018-07-02: (array([76.99935791]), 43.11735617365377)', 'interval: 27, train_start: 2018-01-01, train_end: 2018-07-09: (array([81.25511843]), 44.0228364860092)', 'interval: 28, train_start: 2018-01-01, train_end: 2018-07-16: (array([66.34123728]), 43.669958906296216)']
making date ranges...
copying dataframes...
running lacer...
preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

output=['interval: 21, train_start: 2018-01-01, train_end: 2018-05-28: (array([95.16250911]), 43.46016518949431)', 'interval: 22, train_start: 2018-01-01, train_end: 2018-06-04: (array([96.61961472]), 44.145179889437806)', 'interval: 23, train_start: 2018-01-01, train_end: 2018-06-11: (array([95.67498684]), 43.74815735594257)', 'interval: 24, train_start: 2018-01-01, train_end: 2018-06-18: (array([110.56939455]), 44.326671874930646)', 'interval: 25, train_start: 2018-01-01, train_end: 2018-06-25: (array([77.78828816]), 43.859590175430206)', 'interval: 26, train_start: 2018-01-01, train_end: 2018-07-02: (array([76.99935791]), 43.11735617365377)', 'interval: 27, train_start: 2018-01-01, train_end: 2018-07-09: (array([81.25511843]), 44.0228364860092)', 'interval: 28, train_start: 2018-01-01, train_end: 2018-07-16: (array([66.34123728]), 43.669958906296216)', 'interval: 29, train_start: 2018-01-01, train_end: 2018-07-23: (array([60.55904899]), 43.569600541772765)']


In [5]:
def cer_all(df, df1, train_start_date, train_end_date, test_start_date, test_end_date, predictor_num):
    print("preprocessing...")
    #Create Training and Testing Sets
    dftrain = preprocessing(df , train_start_date, train_end_date)
    dftrain = dftrain.reset_index(drop = True)
    dftest = preprocessing(df1, test_start_date, test_end_date)
    dftest = dftest.reset_index(drop = True)

    #Reserve test set for training on all 3 models. 
    y_train, y_test = CreateTestSet(dftest, predictor_num)
    y_test = y_test.reshape((-1, 1))
    output = []
    for request_type in pd.unique(df['RequestType']):
        for cd in range(1,16):
            print(f"running for {str(request_type)} and {str(cd)}...")
            this_metric = str(lacer(dftrain, dftest, y_train, y_test, train_start_date, train_end_date, test_start_date, test_end_date,
                                        request_type, cd, predictor_num))
            output.append(str(request_type) + ', ' + str(cd) + ': ' + this_metric)
            print(f"output={output}")
    return output

In [6]:
cer_all_list = cer_all(df, df1, "2018-01-01", "2018-06-01", "2018-06-02", "2018-09-02", 50)

preprocessing...


/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andyhan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

running for Other and 1...


100% |########################################################################|
100% |########################################################################|


output=['Other, 1: (array([568.9499894]), 138.98630893730746)']
running for Other and 2...


...

output=['Other, 1: (array([568.9499894]), 138.98630893730746)', 'Other, 2: (array([568.71242369]), 139.7341780994744)', 'Other, 3: (array([568.61112747]), 136.57760012442992)', 'Other, 4: (array([568.97199083]), 141.77622806329472)', 'Other, 5: (array([569.5513475]), 142.59937440812388)', 'Other, 6: (array([568.90505901]), 137.329913628425)', 'Other, 7: (array([569.92425905]), 141.2307515249559)', 'Other, 8: (array([568.76547834]), 138.45626443718209)', 'Other, 9: (array([569.30870036]), 135.5334747060099)', 'Other, 10: (array([569.16292697]), 142.00729772436708)', 'Other, 11: (array([569.18961651]), 141.71071744434036)', 'Other, 12: (array([611.50849543]), 165.1828592889345)', 'Other, 13: (array([569.12512594]), 141.13788080246758)', 'Other, 14: (array([569.36727388]), 138.83709589593033)', 'Other, 15: (array([568.68912555]), 136.23943952969876)', 'Graffiti Removal, 1: (array([567.85315628]), 134.75583185638857)', 'Graffiti Removal, 2: (array([567.92281671]), 135.6076704188285)', 'Gra

100% |########################################################################|
100% |########################################################################|


output=['Other, 1: (array([568.9499894]), 138.98630893730746)', 'Other, 2: (array([568.71242369]), 139.7341780994744)', 'Other, 3: (array([568.61112747]), 136.57760012442992)', 'Other, 4: (array([568.97199083]), 141.77622806329472)', 'Other, 5: (array([569.5513475]), 142.59937440812388)', 'Other, 6: (array([568.90505901]), 137.329913628425)', 'Other, 7: (array([569.92425905]), 141.2307515249559)', 'Other, 8: (array([568.76547834]), 138.45626443718209)', 'Other, 9: (array([569.30870036]), 135.5334747060099)', 'Other, 10: (array([569.16292697]), 142.00729772436708)', 'Other, 11: (array([569.18961651]), 141.71071744434036)', 'Other, 12: (array([611.50849543]), 165.1828592889345)', 'Other, 13: (array([569.12512594]), 141.13788080246758)', 'Other, 14: (array([569.36727388]), 138.83709589593033)', 'Other, 15: (array([568.68912555]), 136.23943952969876)', 'Graffiti Removal, 1: (array([567.85315628]), 134.75583185638857)', 'Graffiti Removal, 2: (array([567.92281671]), 135.6076704188285)', 'Gra

100% |########################################################################|
100% |########################################################################|


output=['Other, 1: (array([568.9499894]), 138.98630893730746)', 'Other, 2: (array([568.71242369]), 139.7341780994744)', 'Other, 3: (array([568.61112747]), 136.57760012442992)', 'Other, 4: (array([568.97199083]), 141.77622806329472)', 'Other, 5: (array([569.5513475]), 142.59937440812388)', 'Other, 6: (array([568.90505901]), 137.329913628425)', 'Other, 7: (array([569.92425905]), 141.2307515249559)', 'Other, 8: (array([568.76547834]), 138.45626443718209)', 'Other, 9: (array([569.30870036]), 135.5334747060099)', 'Other, 10: (array([569.16292697]), 142.00729772436708)', 'Other, 11: (array([569.18961651]), 141.71071744434036)', 'Other, 12: (array([611.50849543]), 165.1828592889345)', 'Other, 13: (array([569.12512594]), 141.13788080246758)', 'Other, 14: (array([569.36727388]), 138.83709589593033)', 'Other, 15: (array([568.68912555]), 136.23943952969876)', 'Graffiti Removal, 1: (array([567.85315628]), 134.75583185638857)', 'Graffiti Removal, 2: (array([567.92281671]), 135.6076704188285)', 'Gra

In [9]:
pprint.pprint(cer_all_list)

['Other, 1: (array([568.9499894]), 138.98630893730746)',
 'Other, 2: (array([568.71242369]), 139.7341780994744)',
 'Other, 3: (array([568.61112747]), 136.57760012442992)',
 'Other, 4: (array([568.97199083]), 141.77622806329472)',
 'Other, 5: (array([569.5513475]), 142.59937440812388)',
 'Other, 6: (array([568.90505901]), 137.329913628425)',
 'Other, 7: (array([569.92425905]), 141.2307515249559)',
 'Other, 8: (array([568.76547834]), 138.45626443718209)',
 'Other, 9: (array([569.30870036]), 135.5334747060099)',
 'Other, 10: (array([569.16292697]), 142.00729772436708)',
 'Other, 11: (array([569.18961651]), 141.71071744434036)',
 'Other, 12: (array([611.50849543]), 165.1828592889345)',
 'Other, 13: (array([569.12512594]), 141.13788080246758)',
 'Other, 14: (array([569.36727388]), 138.83709589593033)',
 'Other, 15: (array([568.68912555]), 136.23943952969876)',
 'Graffiti Removal, 1: (array([567.85315628]), 134.75583185638857)',
 'Graffiti Removal, 2: (array([567.92281671]), 135.607670418828

CD district 12 has consistently high error

TODO: do statistical analysis (standard deviation) on y_predFinal